In [3]:
from PIL import Image
import numpy as np
import colorsys
import random

def match(img1,img2,factor=1):
    w1, h1 = img1.size
    w2, h2 = img2.size
    img2 = img2.resize((w1, h1))

    return img1.reduce(factor), img2.reduce(factor)

def gridtoarr(grid):
    arr = []
    for i in range(len(grid)):
        for j in grid[i]:
            arr.append(j)
    return np.array(arr)

def arrtogrid(arr, width, height):
    grid = []
    for i in range(height):
        row = []
        for j in range(width):
            row.append(arr[i * width + j])
        grid.append(row)
    return np.array(grid)

def imgrgb(img):
    rgb = img.convert('RGB')
    grid = np.array(rgb)  # shape (H, W, 3), dtype=uint8
    return gridtoarr(grid)

def imgl(img):
    l = img.convert('L')
    grid = np.array(l)  # shape (H, W), dtype=uint8
    return gridtoarr(grid)

def randise(arr, v1, v2, v3, range1, range2, range3):
    arr_new = []
    for i in arr:
        r, g, b = i
        r_new = random.randint(0,range1) if v1 else r
        g_new = random.randint(0,range2) if v2 else g
        b_new = random.randint(0,range3) if v3 else b
        arr_new.append([r_new, g_new, b_new])
    return np.array(arr_new, dtype=np.uint8)

def image_from_rgb(rgb_arr):
    return Image.fromarray(rgb_arr, mode='RGB')


#image1: target, image2: pixels
image1 = Image.open('pythonimg/obama.jpg')
image2 = Image.open('pythonimg/noise.png')
image1, image2 = match(image1, image2)


randomiser = 0
saved = 1


if randomiser:
    arr1 = imgrgb(image1)
    randarr = randise(arr1, 0, 1, 1, 255, 255, 255)
    randimage1 = image_from_rgb(arrtogrid(randarr, image1.size[0], image1.size[1]))
    randimage1.show()
    
else:
    arr1temp = imgl(image1)
    arr2temp = imgl(image2)
    n = len(arr1temp)
    print(n)                                   
    
    arr1 = [[arr1temp[i],i] for i in range(n)]
    arr2 = [[arr2temp[i],i] for i in range(n)]
    
    arr1s = sorted(arr1, key=lambda x: x[0])
    arr2s = sorted(arr2, key=lambda x: x[0])

    rgb2 = imgrgb(image2)
    arrf = [0]*n
    for i in range(n):
        targ = arr1s[i][1]
        pix = arr2s[i][1]
        arrf[targ] = rgb2[pix]

    imagef = image_from_rgb(arrtogrid(np.array(arrf, dtype=np.uint8), image1.size[0], image1.size[1]))
    image2.show()
    imagef.show()
    if saved:
        imagef.save('output.png')




50625


Unused code

In [ ]:
#unused functions

# ind1 = [arr1s[i][1] for i in range(n)]
# ind2 = [arr2s[i][1] for i in range(n)]
# print("c2")

# indf = [ind2[ind1.index(i)] for i in range(n)]

# print("c3")


# arrf = [rgb2[indf[i]] for i in range(n)]

def channel(arr,a):
    arr_new = []
    for i in arr:
        arr_new.append(i[a])
    return np.array(arr_new, dtype=np.uint8)


def rgb_to_hls(pixels_rgb):
    pixels_hls = []
    for i in pixels_rgb:
        r, g, b = i
        r_norm, g_norm, b_norm = r / 255.0, g / 255.0, b / 255.0
        h, l, s = colorsys.rgb_to_hls(r_norm, g_norm, b_norm)
        pixels_hls.append((h, l, s))

    hsl_array = np.array(pixels_hls) 
    return hsl_array

def hls_array_to_rgb_array_colorsys(hls_array):
    # Create an empty array for RGB values
    rgb_array = np.empty_like(hls_array)

    # Iterate over each pixel and convert
    # This can be slow for large images
    it = np.nditer(hls_array, flags=['multi_index'], op_flags=['readonly'])
    for x in it:
        idx = it.multi_index
        h, l, s = hls_array[idx]
        r, g, b = colorsys.hls_to_rgb(h, l, s)
        rgb_array[idx] = [r, g, b]

    return rgb_array

Everything below this shows testing

In [ ]:
from PIL import Image
import numpy as np
import colorsys

def match(img1,img2):
    w1, h1 = img1.size
    w2, h2 = img2.size

    htarget = min(h1,h2)
    print((w1, h1),(w2,h2))
    
    img1 = img1.resize((w1*htarget//h1,htarget))
    img2 = img2.resize((w2*htarget//h2,htarget))
    w1, h1 = img1.size
    w2, h2 = img2.size
    print((w1, h1),(w2,h2))

    wtarget = min(w1,w2)
    img1 = img1.crop(((w1-wtarget)//2, 0, w1-((w1-wtarget)//2 + (w1-wtarget)%2), htarget))
    img2 = img2.crop(((w2-wtarget)//2, 0, w2-((w2-wtarget)//2 + (w2-wtarget)%2), htarget))

    return img1, img2

image1 = Image.open('pythonimg/obama.jpg')
image2 = Image.open('pythonimg/sweeney.jpg')
image1, image2 = match(image1, image2)
print(image1.size, image2.size)
image1.show()
image2.show()


(225, 225) (300, 168)
(168, 168) (300, 168)
(168, 168) (168, 168)


In [ ]:
def minim(a, b):
    asort = sorted(a)
    bsort = sorted(b)
    
    bmin = []
    for i in range(n):
        ind = asort.index(a[i])
        bmin.append(bsort[ind])
        asort[ind] = -1
        bsort[ind] = -1
    return bmin

In [ ]:
from PIL import Image
import numpy as np
import colorsys

def match_and_crop(img1, img2, resample=Image.LANCZOS):

    w1, h1 = img1.size
    w2, h2 = img2.size

    if (w1 > h1) != (w2 > h2):
        img2 = img2.rotate(90, expand=True)
        w2, h2 = img2.size

    target_w = min(w1, w2)
    target_h = min(h1, h2)

    def scale_and_center_crop(img, target_w, target_h):
        w, h = img.size
        # Compute scale so image is >= target in both dims (scale <= 1 so we only shrink)
        scale = max(target_w / w, target_h / h)
        new_w = max(1, int(round(w * scale)))
        new_h = max(1, int(round(h * scale)))
        if (new_w, new_h) != (w, h):
            img = img.resize((new_w, new_h), resample=resample)
        left = (new_w - target_w) // 2
        top = (new_h - target_h) // 2
        right = left + target_w
        bottom = top + target_h
        return img.crop((left, top, right, bottom))

    img1_fixed = scale_and_center_crop(img1, target_w, target_h)
    img2_fixed = scale_and_center_crop(img2, target_w, target_h)
    return img1_fixed, img2_fixed

# Example usage: load images and produce matched-size versions (never 0x0)
image1 = Image.open('pythonimg/sweeney.jpg')
image2 = Image.open('pythonimg/obama.jpg')
image1, image2 = match_and_crop(image1, image2)
print(image1.size, image2.size)
image1.show()
image2.show()



(225, 168) (225, 168)


In [ ]:
import random
import itertools

n = 7

arr1 = [random.randint(1,4) for _ in range(n)]
arr2 = [random.randint(1,4) for _ in range(n)]

def min1(a, b):
    arrlist = list(itertools.permutations(b,n))
    lossarr = []
    for i in range(len(arrlist)):
        loss = 0
        for j in range(n):
            loss += (a[j]-arrlist[i][j])**2
        lossarr.append(loss)
    ind = lossarr.index(min(lossarr))
    return list(arrlist[ind])

def swap(a1,a2,b1,b2):
    loss1 = (a1 - b1) ** 2 + (a2 - b2) ** 2
    loss2 = (a1 - b2) ** 2 + (a2 - b1) ** 2
    return loss2 < loss1

def min2(a, b):
    asort = sorted(a)
    bsort = sorted(b)
    k = 1
    while k:
        k = 0
        for i in range(n-1):
            if swap(asort[i], asort[i+1], bsort[i], bsort[i+1]):
                bsort[i], bsort[i+1] = bsort[i+1], bsort[i]
                k = 1
    
    bmin = []
    for i in range(n):
        ind = asort.index(a[i])
        bmin.append(bsort[ind])
        asort[ind] = -1
        bsort[ind] = -1
    return bmin

def min3(a, b):
    asort = sorted(a)
    bsort = sorted(b)
    
    bmin = []
    for i in range(n):
        ind = asort.index(a[i])
        bmin.append(bsort[ind])
        asort[ind] = -1
        bsort[ind] = -1
    return bmin

def loss(a, b):
    tot = 0
    for i in range(n):
        tot += (a[i]-b[i])**2
    return tot

arr2min1 = min1(arr1, arr2)

arr2min2 = min2(arr1, arr2)

arr2min3 = min3(arr1, arr2)


print(loss(arr1,arr2min1)==loss(arr1,arr2min2), loss(arr1,arr2min1)==loss(arr1,arr2min3))


True True
